In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

%matplotlib inline

plt.style.use('default') # Make the graphs a bit prettier

plt.rcParams['figure.figsize'] = (12, 4)

In [2]:
#pip install -U statsmodels
import statsmodels.api as sm 

/usr/local/lib/python2.7/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


#  Preprocesamiento de Datos

El analisis exploratorio que se presenta a continuacion tiene como fin determinar que feathers influyen en mayor medida a la hora de determinar el precio de un inmueble en Capital Federal y Gran Buenos Aires, asi como tambien encontrar datos curiosos en el data set.

Los datos fueron provistos por [Properati](http://www.properati.com.ar) y reorganizo en un solo dataset de la siguiente manera : [GitHub | properati_argentina](https://github.com/BraianVicente/properati_argentina/blob/master/src/TP1%20-%20Properati%20-%20Parseo%20de%20datos%20por%20fecha.ipynb)

In [3]:
sell_properati = pd.read_csv('../data/properati_full_month.csv',low_memory=False)

Transformando el formato del feature created_on para manejarlo como fecha

In [4]:
sell_properati['created_on'] = sell_properati['created_on'].apply(lambda x : pd.to_datetime(x))

In [5]:
sell_properati['year'] = sell_properati['created_on'].apply(lambda x : x.year)

In [6]:
def parse_year_month(date):
    if len(str(date.month)) == 1 :
        return str(date.year) + '-0' + str(date.month)
    return str(date.year) + '-' + str(date.month) 

In [7]:
sell_properati['year_month'] = sell_properati.created_on.apply(parse_year_month)

Completamos la columna 'state_name'

In [8]:
sell_properati['state_name'] = sell_properati.place_with_parent_names.apply(lambda x : x.split('|')[2])

Filtramos unicamente los que nos interesan que corresponde a capital federal y alrededores

In [9]:
sell_properati = sell_properati[
    (sell_properati['state_name'] == 'Bs.As. G.B.A. Zona Norte' ) |
    (sell_properati['state_name'] == 'Bs.As. G.B.A. Zona Sur' ) |
    (sell_properati['state_name'] == 'Bs.As. G.B.A. Zona Oeste' ) |
    (sell_properati['state_name'] == 'Capital Federal' ) 
    ]

In [10]:
sell_properati['zone'] = sell_properati.place_with_parent_names.apply(lambda x : x.split('|')[3])

Filtramos por los datos que son reelevantes para nuestro analisis, obteniendo unicamente las propiedades de GBA y capital federal

In [11]:
sell_properati.drop(inplace=True,\
    labels=['country_name','description','extra','id', u'image_thumbnail','operation'\
            ,'place_with_parent_names_l1','place_with_parent_names_l2', 'properati_url','surface_in_m2','title',\
           ],axis=1)

Podemos seguir trabajando con los datos que no cuentan con la informacion de Zona utilizando la ubicacion para identificarlos segun el barrio al que pertenecen pero en esta ocacion decidimos dejarlos de lado pues no son una cantidad significativa de datos

#### Las propiedades del tigre son las unicas que no contienen informacion en 'place_name'

In [12]:
sell_properati.loc[pd.isnull(sell_properati['place_name']),'place_name'] = sell_properati.place_name.apply(lambda x : x if pd.notnull(x) else 'Tigre' )

In [13]:
sell_properati = sell_properati[sell_properati['zone'] != '']

In [14]:
#Calcula el precio la incognita (incognita = divisor/dividendo) para todo divisor mayor a cero y todo dividendo mayor a cero.

def dataframe_calc_value(df,incognita,divisor,dividendo):
    df.loc[ (pd.isnull(df[incognita])) |  (df[incognita] == 0 ) & (pd.notnull(df[divisor]) ) & (df[divisor] > 0) & \
            (pd.notnull(df[dividendo]) & (df[dividendo] > 0)) ,incognita] = \
    df.loc[ (pd.isnull(df[incognita])) |  (df[incognita] == 0 ) & (pd.notnull(df[divisor]) ) & (df[divisor] > 0) & \
            (pd.notnull(df[dividendo]) & (df[dividendo] > 0))][divisor] / \
    df.loc[ (pd.isnull(df[incognita])) |  (df[incognita] == 0 ) & (pd.notnull(df[divisor]) ) & (df[divisor] > 0) & \
            (pd.notnull(df[dividendo]) & (df[dividendo] > 0))][dividendo]

# Continuaremos trabajando con el set de datos completo para disponer de toda la informacion.

### Se agrega la superficie total de los inmbuebles que disponen del precio por metro cuadrado en dolares


In [15]:
sell_properati_filter = sell_properati

In [16]:
dataframe_calc_value(sell_properati_filter,'surface_total_in_m2','price_aprox_usd','price_usd_per_m2')

### Calculamos la superficie total en metros cuadrados de las propiedades que no cuentan con este dato

In [17]:
dataframe_calc_value(sell_properati_filter,'surface_total_in_m2','price','price_per_m2')

### Calculamos el precio por metro cuadrado en USD de los inmuebles que no cuentan con dicha informacion

In [18]:
dataframe_calc_value(sell_properati_filter,'price_usd_per_m2','price_aprox_usd','surface_total_in_m2')

### Se agrega el precio por metro cuadrado en la moneda actual en cada unos de las pripiedades

In [19]:
dataframe_calc_value(sell_properati_filter,'price_per_m2','price','surface_total_in_m2')

In [20]:
sell_properati_filter = sell_properati_filter[~((sell_properati_filter['surface_total_in_m2'] == 0 ) | \
                     (sell_properati_filter['price'] == 0 ))]

In [21]:
sell_properati_filter[['surface_total_in_m2','price_usd_per_m2','price_per_m2']].mean()

surface_total_in_m2    7402.093163
price_usd_per_m2       2268.710530
price_per_m2           3892.322905
dtype: float64

In [22]:
sell_properati_filter[(pd.notnull(sell_properati_filter['surface_total_in_m2']))]\
            [['surface_total_in_m2','price_usd_per_m2','price_per_m2','price_aprox_usd','price']].shape

(278094, 5)

Filtraremos los datos que tengan una superficie total en metros cuadrados entre 0 y 1000

In [23]:
filter_properati = sell_properati_filter[(sell_properati_filter['price_aprox_usd'] < 600000) &\
                                         (sell_properati_filter['price_usd_per_m2'] < 5000) & \
                                         (sell_properati_filter['surface_total_in_m2'] < 1000)]

In [24]:
float(filter_properati.shape[0]) / float(sell_properati.shape[0]) * 100

62.02927539329903

In [25]:
filter_properati.columns

Index([u'created_on', u'currency', u'expenses', u'floor', u'geonames_id',
       u'lat', u'lat-lon', u'lon', u'place_name', u'place_with_parent_names',
       u'price', u'price_aprox_local_currency', u'price_aprox_usd',
       u'price_per_m2', u'price_usd_per_m2', u'property_type', u'rooms',
       u'state_name', u'surface_covered_in_m2', u'surface_total_in_m2',
       u'year', u'year_month', u'zone'],
      dtype='object')

In [26]:
data = filter_properati[['floor','expenses','price_aprox_usd','rooms','surface_total_in_m2']].copy()

In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 243749 entries, 2 to 597227
Data columns (total 5 columns):
floor                  47217 non-null float64
expenses               19623 non-null object
price_aprox_usd        243749 non-null float64
rooms                  160435 non-null float64
surface_total_in_m2    243749 non-null float64
dtypes: float64(4), object(1)
memory usage: 11.2+ MB


In [28]:
def devolverInt(x):
    return float(re.search(r'\d+', x).group())

In [29]:
import re
string1 = "$ 1500.- aprox results should get"
int(re.search(r'\d+', string1).group())


1500

In [30]:
devolverInt(string1)

1500.0

In [31]:
data['expenses'] = data.expenses.apply(lambda x : devolverInt(x) if pd.notnull(x) else 0 )

In [32]:
data['floor'] = data.floor.apply(lambda x : float(x) if pd.notnull(x) else 0 )

data['lat'] = data.lat.apply(lambda x : float(x) if pd.notnull(x) else 0 )

data['lon'] = data.lon.apply(lambda x :float(x) if pd.notnull(x) else 0 )

In [33]:
data['price_aprox_usd'] = data.price_aprox_usd.apply(lambda x : float(x) if pd.notnull(x) else 0 )

data['price_usd_per_m2'] = data.price_usd_per_m2.apply(lambda x : float(x) if pd.notnull(x) else 0 )

In [34]:
data['rooms'] = data.rooms.apply(lambda x : float(x) if pd.notnull(x) else 0 )

In [35]:
data['surface_total_in_m2'] = data.surface_total_in_m2.apply(lambda x : float(x) if pd.notnull(x) else 0 )

In [36]:
data.tail()

,floor,expenses,price_aprox_usd,rooms,surface_total_in_m2
597200,0.0,0.0,130000.0,1.0,49.0
597213,0.0,0.0,160000.0,4.0,203.0
597221,0.0,0.0,65000.0,2.0,120.0
597225,0.0,8600.0,550000.0,3.0,324.0
597227,1.0,0.0,450000.0,3.0,268.0


In [47]:
train = data.sample(int(float(data.shape[0]) * 0.8)).copy()

In [48]:
test = data.sample(int(float(data.shape[0]) * 0.1)).copy()

# Machine Learning


In [49]:
train_target = train[['price_aprox_usd']].copy()

In [50]:
train.drop(inplace=True,labels=['price_aprox_usd'],axis=1)

In [51]:
test_target = test[['price_aprox_usd']].copy()

In [52]:
test.drop(inplace=True,labels=['price_aprox_usd'],axis=1)

In [53]:
msqDF = pd.DataFrame(columns={'k-Neighbors','p_minkowski','mean_squear_error'})

In [54]:
from sklearn import neighbors

In [55]:
for k in xrange(1,1+200) :
    print k
    for p in xrange(1,1+5):
        knn = neighbors.KNeighborsRegressor(n_neighbors=k,n_jobs=-1,p=p)
        knn.fit(train,train_target)
        test_prediction = knn.predict(test)
        msq_knn = mean_squared_error(test_prediction, test_target)
        msqDF = msqDF.append( pd.DataFrame(data={'k-Neighbors' : [k],'p_minkowski' : [p] ,'mean_squear_error' :[msq_knn]}) )

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200


In [64]:
msqDF.groupby(by=['k-Neighbors','p_minkowski']).min()

mean_squear_error
k-Neighbors p_minkowski                   
1           1                 6.397702e+09
            2                 6.403227e+09
            3                 6.403227e+09
            4                 6.403463e+09
            5                 6.403442e+09
2           1                 5.460282e+09
            2                 5.454105e+09
            3                 5.453464e+09
            4                 5.454310e+09
            5                 5.453739e+09
3           1                 5.211525e+09
            2                 5.213593e+09
            3                 5.214038e+09
            4                 5.213940e+09
            5                 5.214330e+09
4           1                 5.241415e+09
            2                 5.244483e+09
            3                 5.241457e+09
            4                 5.241210e+09
            5                 5.241432e+09
5           1                 5.328984e+09
            2                 5.337502e+09
            3                 5.335070e+09
            4                 5.336166e+09
            5                 5.336083e+09
6           1                 5.430409e+09
            2                 5.437332e+09
            3                 5.438054e+09
            4                 5.439958e+09
            5                 5.440079e+09
...                                    ...
195         1                 6.470543e+09
            2                 6.476727e+09
            3                 6.473154e+09
            4                 6.471654e+09
            5                 6.471591e+09
196         1                 6.469895e+09
            2                 6.479633e+09
            3                 6.474486e+09
            4                 6.474679e+09
            5                 6.474379e+09
197         1                 6.473663e+09
            2                 6.478212e+09
            3                 6.475014e+09
            4                 6.474545e+09
            5                 6.474453e+09
198         1                 6.474208e+09
            2                 6.483001e+09
            3                 6.479393e+09
            4                 6.479430e+09
            5                 6.478799e+09
199         1                 6.473739e+09
            2                 6.484942e+09
            3                 6.480877e+09
            4                 6.480719e+09
            5                 6.480335e+09
200         1                 6.475058e+09
            2                 6.484893e+09
            3                 6.480362e+09
            4                 6.479914e+09
            5                 6.480155e+09

[1000 rows x 1 columns]

In [57]:
msqDF.describe()

,mean_squear_error
count,1.000000e+03
mean,6.195217e+09
std,2.840578e+08
min,5.211525e+09
25%,6.069047e+09
50%,6.300222e+09
75%,6.403984e+09
max,6.484942e+09


In [60]:
msqDF[msqDF['mean_squear_error'] == msqDF.mean_squear_error.min()]

,k-Neighbors,mean_squear_error,p_minkowski
0,3,5.211525e+09,1


###  Se llega la conclusion de que con un K=3 y usando la distancia Manhatan se obtiene el mejor resultado con este set de datos sin modificaciones y unicamente obteniendo el resultado en base floor	expenses	price_aprox_usd	rooms	surface_total_in_m2